## 04前馈神经网络
___

In [7]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
# 设置你自己的device
device = torch.device('cpu')
# 设置超参数
input_size = 784
hidden_size = 500
num_classes = 10
num_epoches = 5 
batch_size = 100
learning_rate = 0.001

# MNIST数据集
train_dataset = torchvision.datasets.MNIST(root='/home/wangye/data',
                                          train=True,
                                          transform=transforms.ToTensor(),
                                          download=True)
test_dataset = torchvision.datasets.MNIST(root='/home/wangye/data',
                                         train=False,
                                         transform=transforms.ToTensor())
# 数据加载
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)

In [8]:
# 全连接神经网络、一个隐藏层
class NeuralNet(nn.Module):
    def __init__(self,input_size,hidden_size,num_classes):
        super(NeuralNet,self).__init__()
        self.fc1 = nn.Linear(input_size,hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size,num_classes)
    def forward(self,x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out 

## 代码讲解
___
1. super(NeuralNet,self).__init__()
    * 我们定义了自己的网络类NeualNet,并且我们继承了nn.Moudle，那么必须在子类的构造函数中声明，也就是上述代码的意义
    * 本网络设置了两个层，fc1，fc2，并且在fc1和fc2之间设置了Relu激活函数
2. __init__():
    * 在类初始化的时候便会执行，基本都是用来设置网络结构的
3. forward(* input):
    * 根据官方文档,此函数定义了每次执行的计算步骤，在所有的子类中，我们都需要重写这个函数，也就是上述代码。


In [10]:
# 实例化模型，并指定device
model = NeuralNet(input_size,hidden_size,num_classes).to(device)
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)



## 上述代码讲解：
1. torch.optim.Adam()
    * 其实这也是优化器的一种，只不过这种优化器实现了ADAM算法，不必太过在意定义性的东西，最主要的是学会使用，然后慢慢深入
    

In [11]:
# 训练模型
total_step = len(train_loader)
for epoch in range(num_epoches):
    for i,(images , labels) in enumerate(train_loader):
        images = images.reshape(-1,28*28).to(device)
        labels = labels.to(device)
        
        
#       向前传递
        outputs = model(images)
        loss = criterion(outputs,labels)
#       反向传播和优化参数
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1)%100 == 0:
#             训练100条，便输出具体训练情况
            print('Epoch[{}/{}],Step[{}/{}],Loss:{:.4f}'.format(
            epoch+1,num_epoches,i+1,total_step,loss.item()))

Epoch[1/5],Step[100/600],Loss:0.3001
Epoch[1/5],Step[200/600],Loss:0.2607
Epoch[1/5],Step[300/600],Loss:0.2079
Epoch[1/5],Step[400/600],Loss:0.2636
Epoch[1/5],Step[500/600],Loss:0.0783
Epoch[1/5],Step[600/600],Loss:0.2077
Epoch[2/5],Step[100/600],Loss:0.1822
Epoch[2/5],Step[200/600],Loss:0.1418
Epoch[2/5],Step[300/600],Loss:0.0752
Epoch[2/5],Step[400/600],Loss:0.0588
Epoch[2/5],Step[500/600],Loss:0.0939
Epoch[2/5],Step[600/600],Loss:0.1257
Epoch[3/5],Step[100/600],Loss:0.0326
Epoch[3/5],Step[200/600],Loss:0.1178
Epoch[3/5],Step[300/600],Loss:0.1635
Epoch[3/5],Step[400/600],Loss:0.0557
Epoch[3/5],Step[500/600],Loss:0.0312
Epoch[3/5],Step[600/600],Loss:0.1455
Epoch[4/5],Step[100/600],Loss:0.0961
Epoch[4/5],Step[200/600],Loss:0.0525
Epoch[4/5],Step[300/600],Loss:0.0449
Epoch[4/5],Step[400/600],Loss:0.0776
Epoch[4/5],Step[500/600],Loss:0.0317
Epoch[4/5],Step[600/600],Loss:0.0136
Epoch[5/5],Step[100/600],Loss:0.0401
Epoch[5/5],Step[200/600],Loss:0.0358
Epoch[5/5],Step[300/600],Loss:0.0121
E

In [24]:
# 测试模型
with torch.no_grad():
    correct = 0 
    total = 0
    for images , labels in test_loader:
        images = images.reshape(-1,28*28).to(device)
        labels = labels.to(device)
#         print(labels.shape)
        outputs = model(images)
#         print(outputs.data.size())
        _,predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        correct += (predicted==labels).sum().item()
    print('10000张测试图片经过该神经网络的测试效果：{}%'.format(100*correct/total))
torch.save(model.state_dict(),'model.ckpt')

10000张测试图片经过该神经网络的测试效果：97.74%


## 上述代码讲解：
1. with torch.no_grad():
    * 在上一篇我们已经提过，在过去我们使用的是“volatile=True”，从当前运行的网络中分离参数，免去梯度运算。那么在新的版本的Pytorch中，我们使用的是上述方法。
2. torch.max(outputs.data,1):
    * 这个方法有设置参数和不设置参数两种方法，不设置参数即为求解最大值，设置dim参数，则指定了具体在哪个维度上找max，我们下面通过程序来具体演示：
    

In [20]:
#设置参数为0 指定维度为第一个维度
import numpy 
a = numpy.array([[1,2,3],[2,1,5]])
a = torch.from_numpy(a)
print(torch.max(a,0))

torch.return_types.max(
values=tensor([2, 2, 5]),
indices=tensor([1, 0, 1]))


可以看到，返回值为[2,2,5]，且具体的索引为[1,0,1],也就是按照行来找，最终得到每列元素的最大值，组成一行

In [22]:
# 设置参数为1 指定维度为第二个维度
print(torch.max(a,1))

torch.return_types.max(
values=tensor([3, 5]),
indices=tensor([2, 2]))


可以看到，返回值为[3,5]，且具体的索引为[2,2],也就是按照列来找，最终得到每行元素的最大值，组成一列